In [1]:
from selenium import webdriver
DRIVER_PATH = 'D:/Software/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [2]:
import time
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [3]:
import requests, hashlib, io
from PIL import Image
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [4]:
import os
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [ ]:
search_term = ['German Shepherd', 'Labrador Retriever', 'Golden Retriever', 'French Bulldog', 'Siberian Husky', 'Dalmatian', 'Shiba Inu', 'Samoyed', 'Cane Corso', 'Goldendoodle']
number_images = 100
for term in search_term:
    search_and_download(search_term=term, driver_path=DRIVER_PATH, number_images=number_images, target_path='./images')

In [ ]:
term = 'Goldendoodle'
number_images = 150
search_and_download(search_term=term, driver_path=DRIVER_PATH, number_images=number_images, target_path='./images')

Found: 100 search results. Extracting links from 0:100
Found: 150 image links, done!
SUCCESS - saved https://minidoodledogs.com/wp-content/uploads/2017/11/Teacup-Goldendoodle-Utah-Idaho-New_York-Philadephia-Massachussets-Georgia-Virginai-California-Washington-Goldendoodle-Puppies-Golden-doodle-Dog-0001-1.jpg - as ./images\goldendoodle\95c5826ce6.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQlpEH9p_2TERMv56zlkzz4sS3ixnDMlCv5Aw&usqp=CAU - as ./images\goldendoodle\efe4fd1964.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTjq3KwZm-YijMY14UDAMET3jyBXFTH71vSMA&usqp=CAU - as ./images\goldendoodle\d963604237.jpg
SUCCESS - saved https://www.trendingbreeds.com/wp-content/uploads/2019/07/Do-Goldendoodles-Like-To-Cuddle-large.jpg - as ./images\goldendoodle\b04521f50a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSdqeTDQvJ9spXyTQC6_JZMzwgUSBTjoXFMqw&usqp=CAU - as ./images\goldendoodle\b611df062c.jpg
SUCCESS - saved 

ERROR - Could not save https://loverdoodles.com/wp-content/uploads/2019/08/Goldendoodle2.png - cannot identify image file <_io.BytesIO object at 0x0000027E6F162AE0>
SUCCESS - saved https://uptownpuppies.com/wp-content/uploads/2017/05/standard-1.jpg - as ./images\goldendoodle\b412722597.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRu7O9WIEp4iubwgauXpRGlY3Bqu_9F4vu0BA&usqp=CAU - as ./images\goldendoodle\e4dccada76.jpg
SUCCESS - saved https://www.k9web.com/wp-content/uploads/2019/03/Miniature-Goldendoodle-7.jpg - as ./images\goldendoodle\1e639496d5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQSxY1mAFQcovLCXfpCEhRg9_1sUb9O6bYbEA&usqp=CAU - as ./images\goldendoodle\84d0603c9d.jpg
SUCCESS - saved https://i.pinimg.com/originals/16/71/98/167198b6813204933f5c682925d0c08b.jpg - as ./images\goldendoodle\0bbd980041.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSEQA94cZFoof3QNjrz1ZsAGHFVT_2bq8LUjA&usqp=CAU - as .